In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../')
import pickle as pkl
import numpy as np
from copy import deepcopy
import mne
import seaborn as sns
import matplotlib.pyplot as plt
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet import forward

plot_params = dict(surface='white', hemi='both', verbose=0)

In [2]:
info = forward.get_info()
info['sfreq'] = 100
fwd = forward.create_forward_model(info=info)
fwd_free = forward.create_forward_model(info=info, fixed_ori=False)


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.4s remaining:    4.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    4.7s remaining:    2.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    4.8s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    1.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(

In [21]:
n_samples = 5

duration_of_trial = (0.0001, 0.001)
settings = dict(duration_of_trial=duration_of_trial, method='standard')

sim_lstm = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=n_samples)

Simulate Source
Simulating data based on sparse patches.


  0%|          | 0/5 [00:00<?, ?it/s]

0
0
0
0
0
Converting Source Data to mne.SourceEstimate object


  0%|          | 0/5 [00:00<?, ?it/s]


Project sources to EEG...

Create EEG trials with noise...


  0%|          | 0/5 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


In [ ]:
import tensorflow as tf
epochs = 10
patience = 10
activation_function = 'relu'
def combi(y_true, y_pred):
    error_1 = tf.keras.losses.CosineSimilarity()(y_true, y_pred)
    error_2 = tf.keras.losses.MeanSquaredError() (y_true, y_pred)
    return error_1 + error_2
# loss = "mean_squared_error" #combi # 'mean_squared_error'
loss = combi
dropout = 0.2
optimizer = tf.keras.optimizers.Adam()
model_params = dict(activation_function=activation_function, n_lstm_layers=2, 
    n_lstm_units=60, n_dense_layers=0, 
    model_type='v2')
train_params = dict(epochs=epochs, patience=patience, tensorboard=True, 
    dropout=0.1, loss=loss, optimizer=optimizer, return_history=True,
    metrics=[], batch_size=8, device='/CPU:0')
net_lstm = Net(fwd, **model_params)
_, history_lstm = net_lstm.fit(sim_lstm, **train_params)

In [ ]:
sim_lstm_test = Simulation(fwd, info, verbose=True, settings=settings).simulate(n_samples=1)
# prediction = net_lstm.predict(sim_lstm_test)

In [10]:
import mne
pth = r"C:\Users\lukas\Dokumente\projects\pcc_neurofeedback\data\pilot_20211213\sound_left.vhdr"
raw = mne.io.read_raw_brainvision(pth, preload=True)
montage = mne.channels.make_standard_montage("standard_1020")
raw.set_montage(montage)

Extracting parameters from C:\Users\lukas\Dokumente\projects\pcc_neurofeedback\data\pilot_20211213\sound_left.vhdr...
Setting channel info structure...
Reading 0 ... 212739  =      0.000 ...   212.739 secs...


C:\Users\lukas\AppData\Local\Temp/ipykernel_4828/2139641247.py:3: RuntimeWarning: No coordinate information found for channels ['VEOG', 'Photodiode']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(pth, preload=True)
C:\Users\lukas\AppData\Local\Temp/ipykernel_4828/2139641247.py:3: RuntimeWarning: Online software filter detected. Using software filter settings and ignoring hardware values
  raw = mne.io.read_raw_brainvision(pth, preload=True)
C:\Users\lukas\AppData\Local\Temp/ipykernel_4828/2139641247.py:3: RuntimeWarning: Channels contain different highpass filters. Lowest (weakest) filter setting (0.00 Hz) will be stored.
  raw = mne.io.read_raw_brainvision(pth, preload=True)
C:\Users\lukas\AppData\Local\Temp/ipykernel_4828/2139641247.py:3: RuntimeWarning: Channels contain different lowpass filters. Highest (weakest) filter setting (500.00 Hz, Nyquist limit) will be stored.
  raw = mne.io.read_raw_brainvision(pt

<RawBrainVision | sound_left.eeg, 33 x 212740 (212.7 s), ~53.6 MB, data loaded>

In [15]:
idc = mne.channel_indices_by_type(raw.info, picks=['eeg'])["eeg"]

raw.ch_names[idc]

TypeError: list indices must be integers or slices, not list

In [20]:
np.array(raw.ch_names)[idc]
# type(idc)

array(['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5',
       'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10',
       'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8',
       'Fp2'], dtype='<U10')